In [ ]:
#creditApprovalUCI -> dropna - > 범주형컬럼만 선택
import pandas as pd

credit = pd.read_csv('creditApprovalUCI.csv')
credit.dropna(inplace=True)
credit.select_dtypes(include='object').A1.replace(['b','a'],[0,1])
tmp = credit.select_dtypes(include='object').A1
pd.get_dummies(tmp).astype("int")      #One Hot Encoding, OHE
pd.get_dummies(credit)

#credit에서 범주형변수들을 OHE -> 1, 0으로 바꾸시고, 기존 수치형 변수랑 합쳐보세요.
tmp1 = pd.get_dummies( credit.select_dtypes(include='object')).astype("int")
tmp2 = credit.select_dtypes(exclude='object')
pd.concat([tmp1, tmp2], axis=1)

pd.get_dummies(credit.A5, drop_first=True)

tmp3 = pd.get_dummies(credit.A6)
import numpy as np
np.argmax( tmp3, axis=1 )

In [102]:
!unzip mortDefault.zip

Archive:  mortDefault.zip
  inflating: mortDefault2007.csv     
  inflating: mortDefault2008.csv     
  inflating: mortDefault2009.csv     


In [103]:
#diabetes.csv를 Outcome1,0일때의 변수 평균
#파티셔닝->train->smote(필요시), DT로 모델링 -> test로 평가
from imblearn.over_sampling import SMOTE
df = pd.read_csv("mortDefault2009.csv")

In [107]:
df.drop("year", axis=1, inplace=True)

In [113]:
df2 = pd.concat([df.query( "default==1"), df.query( "default==0").sample(24741)])

In [116]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df2.drop("default", axis=1), df2["default"], test_size=0.2, random_state=42)
X_train_res, y_train_res = SMOTE(sampling_strategy="auto",k_neighbors=5).fit_resample( X_train, y_train  )

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
dt = DecisionTreeClassifier(max_depth=100)
dt.fit(X_train_res, y_train_res)  #dt.feature_importances_
rf = RandomForestClassifier( max_depth=100)
rf.fit(X_train_res, y_train_res)
ab = AdaBoostClassifier()
ab.fit(X_train_res, y_train_res)


In [ ]:
#secom.zip의 secom가 X변수들, secom_label y변수, 시간정보->datetime
#X변수들의 이름  X1, X2, .....
#  [ 'X' +str(i) for i in range(변수개수)]
#y변수 이름 y
#y -1, 1  -> 0, 1
#파티셔닝, DT, RF, GB으로 모델링 (GB는 GridSearch)
!unzip secom.zip

In [ ]:
X = pd.read_csv("secom.data", sep=" ", header=None)
X.columns =  [ 'X' +str(i+1) for i in range(590)]
y = pd.read_csv("secom_labels.data", sep=" ", header=None)
y.columns = ["y", "date"]
y["y"].replace({-1:0, 1:1}, inplace=True)
y["date"] = pd.to_datetime(y["date"])
df = pd.concat( [X, y], axis=1)

In [ ]:
tmp = df.isnull().mean().sort_values(ascending=False)
cols = tmp[tmp<0.05].index.tolist()
df2 = df[ cols]
df2.dropna(inplace=True)
df2.drop("date", axis=1, inplace=True)

In [172]:
X_train,X_test,y_train,y_test=train_test_split(df2.drop("y", axis=1), df2["y"], test_size=0.1,
                                               random_state=42, stratify = df2["y"])

In [173]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
print( classification_report(y_test, gb.predict(X_test) ))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       130
           1       0.00      0.00      0.00        10

    accuracy                           0.90       140
   macro avg       0.46      0.48      0.47       140
weighted avg       0.86      0.90      0.88       140



In [ ]:
pd.DataFrame( {"cols": df.drop("Outcome", axis=1).columns,
               "imp": rf.feature_importances_}).sort_values("imp", ascending=False).head()

In [76]:
#GridSearchCV   max_depth: 5, 10, 15, cv:5
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV( estimator=dt, param_grid={"max_depth":[5,10,15]}, cv=5)
gs.fit(X_train_res, y_train_res)
gs.best_params_
dt2 = gs.best_estimator_

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_hat = dt.predict( X_test)#dt.predict_proba( X_test) #dt.score( X_test, y_test)
print( classification_report(y_test,y_hat ) )    #confusion_matrix(y_test, y_hat)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_hat = rf.predict( X_test)#dt.predict_proba( X_test) #dt.score( X_test, y_test)
print( classification_report(y_test,y_hat ) )    #confusion_matrix(y_test, y_hat)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_hat = ab.predict( X_test)#dt.predict_proba( X_test) #dt.score( X_test, y_test)
print( classification_report(y_test,y_hat ) )    #confusion_matrix(y_test, y_hat)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_hat = gb.predict( X_test)#dt.predict_proba( X_test) #dt.score( X_test, y_test)
print( classification_report(y_test,y_hat ) )    #confusion_matrix(y_test, y_hat)